# Cleaining de DIM_Tienda

In [11]:
import pandas as pd
import numpy as np

# === 1. Cargar el archivo ===
df_tienda = pd.read_csv("DIM_TIENDA_TEST.csv")

# Copia para limpieza
df_tienda_clean = df_tienda.copy()

# === 2. Calcular medias por entorno para CAJONESESTACIONAMIENTO_NUM, excluyendo ceros ===
mean_cajones = (
    df_tienda_clean
    .loc[df_tienda_clean['CAJONESESTACIONAMIENTO_NUM'] != 0]
    .groupby('ENTORNO_DES')['CAJONESESTACIONAMIENTO_NUM']
    .mean()
)

# === 3. Reemplazar 0 en CAJONESESTACIONAMIENTO_NUM por la media de su ENTORNO_DES
# (excepto si LID_UBICACION_TIENDA es UN_TRAFICO_PEATONAL) ===
cond_cajones_invalidos = (
    (df_tienda_clean['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (df_tienda_clean['LID_UBICACION_TIENDA'] != 'UN_TRAFICO_PEATONAL')
)

df_tienda_clean.loc[cond_cajones_invalidos, 'CAJONESESTACIONAMIENTO_NUM'] = (
    df_tienda_clean.loc[cond_cajones_invalidos, 'ENTORNO_DES']
    .map(mean_cajones).round().fillna(0).astype(int)

    .astype(int)
)

# === 4. Reemplazar 0 en PUERTASREFRIG_NUM por NaN ===
cond_puertas_invalidas = (df_tienda_clean['PUERTASREFRIG_NUM'] == 0)
df_tienda_clean.loc[cond_puertas_invalidas, 'PUERTASREFRIG_NUM'] = np.nan

# === 4.1 Imputar PUERTASREFRIG_NUM usando mediana por ENTORNO_DES ===
medianas_refrig = (
    df_tienda_clean.groupby('ENTORNO_DES')['PUERTASREFRIG_NUM']
    .median()
    .rename("median_refrig")
)

df_tienda_clean = df_tienda_clean.merge(medianas_refrig, on='ENTORNO_DES', how='left')
df_tienda_clean['PUERTASREFRIG_NUM'] = df_tienda_clean['PUERTASREFRIG_NUM'].fillna(df_tienda_clean['median_refrig'])
df_tienda_clean.drop(columns='median_refrig', inplace=True)

# === 5. Reemplazar 0 en MTS2VENTAS_NUM por la media de su ENTORNO_DES ===
mean_mts2 = (
    df_tienda_clean
    .loc[df_tienda_clean['MTS2VENTAS_NUM'] != 0]
    .groupby('ENTORNO_DES')['MTS2VENTAS_NUM']
    .mean()
)

cond_mts2_ceros = df_tienda_clean['MTS2VENTAS_NUM'] == 0
df_tienda_clean.loc[cond_mts2_ceros, 'MTS2VENTAS_NUM'] = (
    df_tienda_clean.loc[cond_mts2_ceros, 'ENTORNO_DES'].map(mean_mts2)
)

# === 6. Imputar SEGMENTO_MAESTRO_DESC faltantes usando moda por entorno ===
modas_por_entorno = (
    df_tienda_clean
    .groupby('ENTORNO_DES')['SEGMENTO_MAESTRO_DESC']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Desconocido')
)

missing_segmentos_before = df_tienda_clean['SEGMENTO_MAESTRO_DESC'].isna().sum()

def imputar_segmento(row):
    if pd.isna(row['SEGMENTO_MAESTRO_DESC']):
        return modas_por_entorno[row['ENTORNO_DES']]
    return row['SEGMENTO_MAESTRO_DESC']

df_tienda_clean['SEGMENTO_MAESTRO_DESC'] = df_tienda_clean.apply(imputar_segmento, axis=1)
missing_segmentos_after = df_tienda_clean['SEGMENTO_MAESTRO_DESC'].isna().sum()

# === 7. Resumen de cambios ===
print("=== Resumen de Limpieza ===")
print(f"Cajones inválidos reemplazados: {cond_cajones_invalidos.sum()}")
print(f"Puertas inválidas reemplazadas por NaN: {cond_puertas_invalidas.sum()}")
print(f"Ceros en MTS2VENTAS_NUM imputados: {cond_mts2_ceros.sum()}")
print(f"Segmentos imputados: {missing_segmentos_before - missing_segmentos_after}")
print("\nMedias de MTS2VENTAS_NUM por ENTORNO_DES:")
print(mean_mts2)

# === 8. Guardar archivo limpio ===
df_tienda_clean.to_csv("cleaned_tienda_test.csv", index=False)


=== Resumen de Limpieza ===
Cajones inválidos reemplazados: 63
Puertas inválidas reemplazadas por NaN: 22
Ceros en MTS2VENTAS_NUM imputados: 30
Segmentos imputados: 0

Medias de MTS2VENTAS_NUM por ENTORNO_DES:
ENTORNO_DES
Base        108.505714
Hogar       109.726596
Peatonal    115.495000
Receso      102.018000
Name: MTS2VENTAS_NUM, dtype: float64


In [12]:
df_tienda_clean.isna().sum()

TIENDA_ID                     0
PLAZA_CVE                     0
NIVELSOCIOECONOMICO_DES       0
ENTORNO_DES                   0
MTS2VENTAS_NUM                0
PUERTASREFRIG_NUM             0
CAJONESESTACIONAMIENTO_NUM    0
LATITUD_NUM                   0
LONGITUD_NUM                  0
SEGMENTO_MAESTRO_DESC         0
LID_UBICACION_TIENDA          0
DATASET                       0
dtype: int64